In [432]:
import numpy as np
import numpy.linalg as LA
import random

In [543]:
# parameters
time_intervals = 60
period = .1
pencil_parameter = 12

## Generating test input

In [544]:
# this is the M value that we won't know with an actual problem
damp_factor = 0
sample_size = 12
frequencies = np.array(random.sample(range(1,15),sample_size))

amplitudes = [complex(random.uniform(-5.0, 5.0), random.uniform(-5.0, 5.0)) for j in range(sample_size)]


In [545]:
# define random wave input
# basically defining Z_1 from the paper
z_0 = np.zeros((sample_size), dtype = complex)
y_list = []
for n in range(sample_size):
    z_0[n] = np.exp( complex(-damp_factor, frequencies[n])*period )

In [547]:
signal_data = np.zeros((time_intervals,sample_size), dtype = complex)
for k in range(time_intervals):
    signal_data[k,:] = z_0**k

In [548]:
# linear comnbination of residues and complex exponentials
# observed_signal is our test input to put into the matrix pencil algorithm
observed_signal = (signal_data@amplitudes)

In [550]:
z_0  #the MPM procedure should solve for these values

array([0.16996714+0.98544973j, 0.45359612+0.89120736j,
       0.82533561+0.56464247j, 0.95533649+0.29552021j,
       0.26749883+0.96355819j, 0.98006658+0.19866933j,
       0.87758256+0.47942554j, 0.76484219+0.64421769j,
       0.36235775+0.93203909j, 0.69670671+0.71735609j,
       0.62160997+0.78332691j, 0.54030231+0.84147098j])

## Matrix Pencil Method (noiselss data version)

In [551]:
# right now the dimensionality gives L eigvals, but we would just truncate to M based off
# sig figs for the noisy case (for noiseless M = L)

def MatrixPencil_noiseless(x, N, L):
#     data list x as input
   
    # define Y2, Y1
    Y1 = np.zeros((N-L,L), dtype=complex)
   
    Y2 = np.zeros((N-L,L), dtype = complex)
    
    for i in range(N-L):
        for j in range(L):
            Y1[i,j] = x[i + j]
            Y2[i,j] = x[i + j + 1]

    #solve eigval problem
    z = LA.eigvals(pseudo_inv(Y1)@Y2)
    
    #return eigval
    return z

In [552]:
def pseudo_inv(A):
    #takes np.array input type, changes to np.matrix, performs comp, returns np.array type
    A = np.matrix(A)
    # assuming columns of A are linearly independent, Moore-Penrose inv takes this form
    return np.array(LA.inv(A.H @ A)@A.H)

In [553]:
eigvals_noiseless = MatrixPencil_noiseless(observed_signal, time_intervals, pencil_parameter)

In [554]:
eigvals_noiseless

array([-1.01478638e-05-1.11159878e-05j,  7.80831147e-02+8.71317908e-01j,
        2.55579224e-01+3.11624593e-01j,  9.54851793e-01+1.01956125e-01j,
        2.68656298e-01+1.01013275e+00j,  3.28368450e-01+8.40635318e-01j,
        3.72650452e-01+8.98368741e-01j,  8.41959165e-01+8.99588346e-01j,
        7.00294746e-01+8.40724084e-01j,  9.46087951e-01+2.91512180e-01j,
        8.38655083e-01+5.04361885e-01j,  7.55403856e-01+6.20418664e-01j])

## Matrix Pencil Method (noisy data procedure)

In [568]:
def MatrixPencil(y, N, L, sigfigs):
#   y - observed time response data
    
    Y = np.zeros((N-L,L+1), dtype = complex)
    
    for i in range(N-L):
        for j in range(L+1): 
            Y[i,j] = y[i+j]

    U,S,Vh = LA.svd(Y, full_matrices = True)

#   print('Y_recons= \n', np.dot(U[:,:S_dim]*S, Vh) )

    threshold = 10**(-sigfigs)
    max_singval = np.max(S)
    
    # define mask to truncate singular values to M
    mask = S/max_singval > threshold
    S[~mask] = 0    
    
    #truncate eigenvectors of V to M dominant ones
    M = np.count_nonzero(mask)
    Vh_trunc = Vh[:M,:]
    
    Vh1 = Vh_trunc[:,:-1]
    Vh2 = Vh_trunc[:,1:]
    
    #solve eigval problem (the papaer takes the pseudo-inverse of Vh1 and Vh2)
    z = LA.eigvals(pseudo_inv(Vh1)@Vh2)
    
    #Solve for residues
    signal_matrix = np.zeros((N,len(z)), dtype = complex)
    for k in range(N):
        signal_matrix[k,:] = z**k
    
#     print('signals= \n',signal_matrix.shape)
#     print('y = \n',y.shape)
    
    residues, _, _, _ = LA.lstsq(signal_matrix, y, rcond = threshold)
    
    # returning z eigvals, residues (complex amplitudes), and signal_matrix (just for checking fidelity)
    return z, residues, signal_matrix

result_eigvals, result_residues, matrix_signal = MatrixPencil(observed_signal, time_intervals, pencil_parameter, sigfigs = 6)



signals= 
 (60, 12)
y = 
 (60,)


In [569]:
result_eigvals

array([ 2.52941371e+00+1.76112449e-01j, -1.16680535e-01+2.26703167e+00j,
        1.94840800e+00+1.82900778e+00j, -3.43803810e-01+6.15044134e-01j,
        2.76725747e-01+1.00748527e+00j,  9.61600842e-01+7.08337329e-02j,
        4.78852089e-01-1.79848731e-01j,  3.35685672e-01+1.05492903e-01j,
        4.61291734e-02+3.16434102e-01j,  9.18242937e-13+2.97140157e-12j,
       -5.50171741e-13-4.45635186e-13j,  2.39172140e-15+4.28678566e-15j])

In [570]:
result_residues

array([ 1.07872694e-22+1.66385754e-23j, -7.02173842e-20+1.04848650e-20j,
        6.97570172e-24+6.94222068e-24j,  2.23945778e-50+1.98527799e-49j,
       -1.67432207e-39+1.71214903e-39j, -3.75050861e-41+7.72421423e-42j,
       -1.05949261e-55+3.59747415e-56j, -8.94218020e-56+2.57894658e-56j,
       -8.11701481e-56+2.27824838e-56j, -8.73440600e-56+2.12829907e-56j,
       -8.68965314e-56+2.08503953e-56j, -8.84824658e-56+1.99059632e-56j])

In [574]:
reconstructed_signal = matrix_signal @ result_residues
reconstructed_signal

array([-7.01025358e-20+1.05084458e-20j, -1.53057006e-20-1.60321045e-19j,
        3.66094090e-19-1.63410310e-20j, -3.54234100e-21+8.31112551e-19j,
       -1.87840366e-18-1.06699927e-19j,  4.74896659e-19-4.24999093e-18j,
        9.61646268e-18+1.56340120e-18j, -4.56616027e-18+2.16017622e-17j,
       -4.81728383e-17-1.28878288e-17j,  3.55175370e-17-1.07663124e-16j,
        2.41584995e-16+9.33835190e-17j, -2.36012366e-16+5.37879502e-16j,
       -1.18279186e-15-5.94734992e-16j,  1.50827103e-15-2.60442828e-15j,
        5.78504370e-15+3.74212999e-15j, -9.00604001e-15+1.27325339e-14j,
       -2.74073280e-14-2.17228318e-14j,  5.34731805e-14-5.89855817e-14j,
        1.29880162e-13+1.30065997e-13j, -3.04902890e-13+2.84904089e-13j,
       -6.00020758e-13-7.09897931e-13j,  1.70068813e-12-1.24295345e-12j,
        2.67069449e-12+4.07944205e-12j, -9.41509212e-12+5.76999389e-12j,
       -1.15621759e-11-2.14885446e-11j,  5.11689370e-11-2.20946235e-11j,
        4.65313335e-11+1.23479359e-10j, -2.81470475

In [567]:
observed_signal

array([  5.47805794+13.70158594j,  -8.02779122 +7.79486183j,
        -4.8068395  -6.03664568j,  10.12093283 -4.80903655j,
        12.47573518+12.26753736j,  -5.23616492+21.15586222j,
       -21.68145601 +7.25133274j, -16.13924739-13.88016591j,
         4.11746183-18.30525817j,  16.21078132 -4.43265519j,
        11.80345259 +9.94780915j,   1.65617257+13.90742342j,
        -4.98919065+12.26484072j, -10.38548102+10.28202351j,
       -16.43044913 +4.6813954j , -17.04747565 -5.67824365j,
        -9.27692572-12.52508638j,  -1.74499885-10.05476302j,
        -2.64769316 -5.80611635j,  -5.93881952 -9.63299218j,
        -0.23477415-17.0230404j ,  12.07908922-14.88246792j,
        16.64163429 -2.32315478j,   8.48503259 +6.46091j   ,
        -0.15146488 +2.71331184j,   2.04361892 -4.4784629j ,
         9.4104005  -2.63716724j,   9.28601649 +5.77204416j,
         1.15276771 +8.5630507j ,  -4.05661395 +2.36662425j,
        -0.59194703 -4.21350956j,   6.28313972 -4.31356712j,
        10.65606322 +0.3

In [575]:
result_eigvals[(np.real(eigvals_general)> 10**(-8))]

array([2.52941371e+00+1.76112449e-01j, 1.94840800e+00+1.82900778e+00j,
       2.76725747e-01+1.00748527e+00j, 9.61600842e-01+7.08337329e-02j,
       4.78852089e-01-1.79848731e-01j, 3.35685672e-01+1.05492903e-01j,
       4.61291734e-02+3.16434102e-01j, 9.18242937e-13+2.97140157e-12j])